In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [4]:
df=pd.read_csv("enhanced_anxiety_dataset.csv")
print("Jumlah record:", df.shape[0])
print("Jumlah kolom:", df.shape[1])
print("\nCek nilai kosong (missing values):")
print(df.isnull().sum())
print("\nJumlah data duplikat:", df.duplicated().sum())

Jumlah record: 11000
Jumlah kolom: 19

Cek nilai kosong (missing values):
Age                                  0
Gender                               0
Occupation                           0
Sleep Hours                          0
Physical Activity (hrs/week)         0
Caffeine Intake (mg/day)             0
Alcohol Consumption (drinks/week)    0
Smoking                              0
Family History of Anxiety            0
Stress Level (1-10)                  0
Heart Rate (bpm)                     0
Breathing Rate (breaths/min)         0
Sweating Level (1-5)                 0
Dizziness                            0
Medication                           0
Therapy Sessions (per month)         0
Recent Major Life Event              0
Diet Quality (1-10)                  0
Anxiety Level (1-10)                 0
dtype: int64

Jumlah data duplikat: 0


In [5]:
numeric_cols = df.select_dtypes(include=['int64','float64']).columns

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    # Cek banyak outlier
    outliers = ((df[col] < lower) | (df[col] > upper)).sum()
    print(f"\nKolom: {col} → Jumlah outlier: {outliers}")



Kolom: Age → Jumlah outlier: 0

Kolom: Sleep Hours → Jumlah outlier: 110

Kolom: Physical Activity (hrs/week) → Jumlah outlier: 56

Kolom: Caffeine Intake (mg/day) → Jumlah outlier: 0

Kolom: Alcohol Consumption (drinks/week) → Jumlah outlier: 0

Kolom: Stress Level (1-10) → Jumlah outlier: 0

Kolom: Heart Rate (bpm) → Jumlah outlier: 0

Kolom: Breathing Rate (breaths/min) → Jumlah outlier: 0

Kolom: Sweating Level (1-5) → Jumlah outlier: 0

Kolom: Therapy Sessions (per month) → Jumlah outlier: 178

Kolom: Diet Quality (1-10) → Jumlah outlier: 0

Kolom: Anxiety Level (1-10) → Jumlah outlier: 322


In [6]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    # Jumlah outlier
    outliers = ((df[col] < lower) | (df[col] > upper)).sum()
    print(f"{col}: {outliers} outlier ditemukan")

    # Hapus outlier
    df = df[(df[col] >= lower) & (df[col] <= upper)]

print("\nDataset setelah menghapus outlier:", df.shape)

Age: 0 outlier ditemukan
Sleep Hours: 110 outlier ditemukan
Physical Activity (hrs/week): 56 outlier ditemukan
Caffeine Intake (mg/day): 0 outlier ditemukan
Alcohol Consumption (drinks/week): 0 outlier ditemukan
Stress Level (1-10): 0 outlier ditemukan
Heart Rate (bpm): 0 outlier ditemukan
Breathing Rate (breaths/min): 0 outlier ditemukan
Sweating Level (1-5): 0 outlier ditemukan
Therapy Sessions (per month): 165 outlier ditemukan
Diet Quality (1-10): 0 outlier ditemukan
Anxiety Level (1-10): 252 outlier ditemukan

Dataset setelah menghapus outlier: (10417, 19)


1. Data Cleaning (Handling Missing Value, Duplicate, dan Outlier)

Tahap data cleaning bertujuan untuk memastikan dataset bersih dan siap dianalisis. Proses ini meliputi penanganan missing value (nilai kosong) yang dapat dihapus atau diganti dengan nilai rata-rata agar tidak menimbulkan bias, penghapusan duplicate record agar tidak terjadi pengulangan data yang dapat memengaruhi hasil analisis, serta penanganan outlier menggunakan metode seperti IQR (Interquartile Range) untuk menghapus data ekstrem yang menyimpang jauh dari distribusi normal. Dengan melakukan pembersihan ini, kualitas dan keandalan data menjadi lebih baik.

In [9]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Buat objek scaler
scaler = MinMaxScaler()

# Lakukan normalisasi
df[num_cols] = scaler.fit_transform(df[num_cols])

print("Data setelah normalisasi:")
print(df.head())


Data setelah normalisasi:
        Age  Gender Occupation  Sleep Hours  Physical Activity (hrs/week)  \
0  0.239130  Female     Artist     0.387097                      0.329268   
1  0.608696   Other      Nurse     0.419355                      0.695122   
2  1.000000    Male      Other     0.225806                      0.451220   
3  0.043478  Female  Scientist     0.354839                      0.341463   
4  0.673913  Female      Other     0.741935                      0.280488   

   Caffeine Intake (mg/day)  Alcohol Consumption (drinks/week) Smoking  \
0                  0.302170                           0.526316     Yes   
1                  0.333890                           0.421053     Yes   
2                  0.195326                           0.210526      No   
3                  0.601002                           0.315789     Yes   
4                  0.412354                           0.210526     Yes   

  Family History of Anxiety  Stress Level (1-10)  Heart Rate (bpm)

2. Normalisasi / Standarisasi Kolom Numerik

Normalisasi atau standarisasi dilakukan untuk menyamakan skala antar fitur numerik sehingga tidak ada satu kolom yang mendominasi analisis atau proses pembelajaran model. Teknik ini biasanya mengubah data agar memiliki rata-rata (mean) = 0 dan standar deviasi = 1. Dengan cara ini, semua fitur numerik berada dalam rentang yang seimbang dan model machine learning dapat belajar secara optimal tanpa bias terhadap variabel dengan nilai besar.

In [13]:
cat_cols = df.select_dtypes(include=['object']).columns
print("Kolom kategorikal:", cat_cols.tolist())

# ======  One-Hot Encoding (paling umum untuk kategori nominal) ======
# drop_first=True untuk menghindari dummy variable trap
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

print("\n Dataset setelah encoding:")
print(df_encoded.head())

# ======  Info tambahan ======
print("\nJumlah kolom sebelum encoding:", len(df.columns))
print("Jumlah kolom sesudah encoding:", len(df_encoded.columns))

Kolom kategorikal: ['Gender', 'Occupation', 'Smoking', 'Family History of Anxiety', 'Dizziness', 'Medication', 'Recent Major Life Event']

 Dataset setelah encoding:
        Age  Sleep Hours  Physical Activity (hrs/week)  \
0  0.239130     0.387097                      0.329268   
1  0.608696     0.419355                      0.695122   
2  1.000000     0.225806                      0.451220   
3  0.043478     0.354839                      0.341463   
4  0.673913     0.741935                      0.280488   

   Caffeine Intake (mg/day)  Alcohol Consumption (drinks/week)  \
0                  0.302170                           0.526316   
1                  0.333890                           0.421053   
2                  0.195326                           0.210526   
3                  0.601002                           0.315789   
4                  0.412354                           0.210526   

   Stress Level (1-10)  Heart Rate (bpm)  Breathing Rate (breaths/min)  \
0             

2. Normalisasi / Standarisasi Kolom Numerik

Normalisasi atau standarisasi dilakukan untuk menyamakan skala antar fitur numerik sehingga tidak ada satu kolom yang mendominasi analisis atau proses pembelajaran model. Teknik ini biasanya mengubah data agar memiliki rata-rata (mean) = 0 dan standar deviasi = 1. Dengan cara ini, semua fitur numerik berada dalam rentang yang seimbang dan model machine learning dapat belajar secara optimal tanpa bias terhadap variabel dengan nilai besar.

In [12]:
print(df.columns)

# =====  Buat fitur baru =====
df['Stress_Index'] = df['Anxiety Level (1-10)'] / df['Sleep Hours']

# =====  Tampilkan hasil =====
print("\n Fitur baru berhasil dibuat:")
print(df[['Anxiety Level (1-10)', 'Sleep Hours', 'Stress_Index']].head())

Index(['Age', 'Gender', 'Occupation', 'Sleep Hours',
       'Physical Activity (hrs/week)', 'Caffeine Intake (mg/day)',
       'Alcohol Consumption (drinks/week)', 'Smoking',
       'Family History of Anxiety', 'Stress Level (1-10)', 'Heart Rate (bpm)',
       'Breathing Rate (breaths/min)', 'Sweating Level (1-5)', 'Dizziness',
       'Medication', 'Therapy Sessions (per month)', 'Recent Major Life Event',
       'Diet Quality (1-10)', 'Anxiety Level (1-10)', 'Stress_Index'],
      dtype='object')

 Fitur baru berhasil dibuat:
   Anxiety Level (1-10)  Sleep Hours  Stress_Index
0                 0.500     0.387097      1.291667
1                 0.250     0.419355      0.596154
2                 0.000     0.225806      0.000000
3                 0.125     0.354839      0.352273
4                 0.000     0.741935      0.000000


4. Feature Engineering (Minimal 1 Fitur Baru)

Feature engineering adalah proses menciptakan fitur baru dari data yang ada untuk meningkatkan kemampuan model dalam mengenali pola. Misalnya, membuat fitur baru bernama Stress_Index dari pembagian antara “Anxiety Level” dan “Sleep Hours” untuk merepresentasikan tingkat stres seseorang. Tahap ini penting karena fitur baru yang informatif dapat meningkatkan akurasi dan interpretabilitas hasil analisis atau prediksi.

In [18]:
target_col = 'Anxiety Level (1-10)'

# =====  Pisahkan fitur (X) dan target (y) =====
X = df.drop(columns=[target_col])
y = df[target_col]

# =====  Split data menjadi training dan testing =====
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,       # 20% data untuk testing
    random_state=42      # agar hasil pembagian selalu sama
)

# ===== Cek hasil =====
print("Jumlah data training:", X_train.shape[0])
print("Jumlah data testing :", X_test.shape[0])
print("\n Data berhasil dibagi menjadi training dan testing.")

Jumlah data training: 8333
Jumlah data testing : 2084

 Data berhasil dibagi menjadi training dan testing.


5. Splitting Data ke dalam Data Training & Data Testing

Pembagian dataset menjadi data training dan testing dilakukan untuk menilai kinerja model secara objektif. Biasanya, 80% data digunakan untuk training agar model dapat belajar pola dari data, sementara 20% sisanya digunakan untuk testing guna menguji seberapa baik model mampu memprediksi data baru yang belum pernah dilihat. Langkah ini penting untuk mencegah overfitting dan memastikan model memiliki kemampuan generalisasi yang baik.